In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics


In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("SUSY")
conf.set("spark.executor.memory", "4G")
conf.set("spark.driver.memory", "20G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 20:47:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/SUSY.csv')
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|                _c10|                _c11|                _c12|                _c13|                _c14|                _c15|                _c16|                _c17|                _c18|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [4]:
data = df.rdd.map(lambda row: LabeledPoint(float(row['_c0']), list(map(lambda x: float(x), row.asDict().values()))[1:]))
# data.take(2)

In [5]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [6]:
# testData.collect()

In [7]:
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=8)

In [8]:
predictions = model.predict(trainingData.map(lambda x: x.features))
predictions

MapPartitionsRDD[58] at mapPartitions at PythonMLLibAPI.scala:1344

In [9]:
labelsAndPredictions = trainingData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] == lp[1]).count() / float(trainingData.count())
print('Test Error = ' + str(testErr))


22/12/03 20:49:13 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 666, in main
    eval_type = read_int(infile)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Ite

22/12/03 20:49:15 WARN TaskSetManager: Lost task 5.0 in stage 23.0 (TID 239) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 0.0 in stage 23.0 (TID 234) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 1.0 in stage 23.0 (TID 235) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 3.0 in stage 23.0 (TID 237) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 6.0 in stage 23.0 (TID 240) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 4.0 in stage 23.0 (TID 238) (c0248 executor driver): TaskKilled (Stage cancelled)
22/12/03 20:49:15 WARN TaskSetManager: Lost task 7.0 in stage 23.0 (TID 241) (c0248 executor driver): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 23.0 failed 1 times, most recent failure: Lost task 2.0 in stage 23.0 (TID 236) (c0248 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/rddsampler.py", line 92, in func
    for obj in iterator:
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/